In [24]:
import os
import openai
import sys

env = {}

# read .env file and set variables
with open('.env') as f:
    for line in f:
        key, value = line.strip().split('=')
        env[key] = value

openai.api_key = env['OPEN_AI_KEY']
OPENAI_API_KEY = env['OPEN_AI_KEY']

In [2]:
llm_name = "gpt-3.5-turbo"

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [5]:
print(vectordb._collection.count())

154


In [6]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)


3

In [25]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key=env['OPEN_AI_KEY'])

In [8]:
from langchain.chains import RetrievalQA

In [9]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [10]:
result = qa_chain({"query": question})

/Users/pikkuaapo/.pyenv/versions/3.11.8/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [11]:
result["result"]

'The major topics for this class include machine learning, control statements (for, while, if statements), plotting data, and computing on data using Octave.'

In [12]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [13]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [14]:
question = "Is probability a class topic?"

In [15]:
result = qa_chain({"query": question})

In [16]:
result["result"]

'Yes, probability is a class topic in the context of machine learning. Thanks for asking!'

In [17]:
result["source_documents"][0]

Document(metadata={'page': 20, 'source': 'docs/Lecture1.pdf'}, page_content='Andrew NgIntroduction \nSupervised Learning \nMachine Learning')

In [18]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [19]:
result = qa_chain_mr({"query": question})

In [20]:
result["result"]

'No, the text provided does not mention probability as a class topic.'

In [21]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [22]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

'Based on the provided context, the class topic is related to machine learning, specifically supervised learning and classifying emails as spam or not spam. Probability is not explicitly mentioned as a class topic in this context.'

In [23]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

"I'm sorry, but I don't have enough information to understand which prerequisites you are referring to. Could you please provide more context or clarify your question?"